<a href="https://colab.research.google.com/github/BryanDanielMartinezCabezas/Inteligencia_Artificial_AI_SIS420/blob/main/LAB04/regresion_one_vs_all_01_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importando librerias**

In [1]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

import pandas as pd

# Libreria para graficos
from matplotlib import pyplot

#Para separa el Dataset 20% y 80% para diferentes pruebas
from sklearn.model_selection import train_test_split

# Modulo de optimizacion en scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Cargando dataset**

In [3]:
data = pd.read_csv('/content/gdrive/MyDrive/Data sete lab 1/FETOS/fetal_health.csv')
data

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.000,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.000,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.000,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140.0,0.000,0.000,0.007,0.000,0.0,0.0,79.0,0.2,25.0,...,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0,2.0
2122,140.0,0.001,0.000,0.007,0.000,0.0,0.0,78.0,0.4,22.0,...,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0,2.0
2123,140.0,0.001,0.000,0.007,0.000,0.0,0.0,79.0,0.4,20.0,...,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0,2.0
2124,140.0,0.001,0.000,0.006,0.000,0.0,0.0,78.0,0.4,27.0,...,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0,2.0


In [5]:
print(pd.isnull(data).sum())

baseline value                                            0
accelerations                                             0
fetal_movement                                            0
uterine_contractions                                      0
light_decelerations                                       0
severe_decelerations                                      0
prolongued_decelerations                                  0
abnormal_short_term_variability                           0
mean_value_of_short_term_variability                      0
percentage_of_time_with_abnormal_long_term_variability    0
mean_value_of_long_term_variability                       0
histogram_width                                           0
histogram_min                                             0
histogram_max                                             0
histogram_number_of_peaks                                 0
histogram_number_of_zeroes                                0
histogram_mode                          

In [6]:
# Definir las variables de entrada (X) y la variable de salida (y)
X = data[['baseline value', 'accelerations', 'fetal_movement',
          'uterine_contractions', 'light_decelerations', 'severe_decelerations',
          'prolongued_decelerations', 'abnormal_short_term_variability',
          'mean_value_of_short_term_variability', 'percentage_of_time_with_abnormal_long_term_variability',
          'mean_value_of_long_term_variability', 'histogram_min', 'histogram_max',
          'histogram_number_of_peaks', 'histogram_number_of_zeroes', 'histogram_mode',
          'histogram_mean', 'histogram_median', 'histogram_variance', 'histogram_tendency']]

y = data['fetal_health']

# Verificar las dimensiones de X y y
print(X.shape)
print(y.shape)


(2126, 20)
(2126,)


In [7]:
y

,fetal_health
0,2.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
2121,2.0
2122,2.0
2123,2.0
2124,2.0


In [8]:
y.value_counts()

,count
fetal_health,
1.0,1655
2.0,295
3.0,176


In [9]:
X

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency
0,120.0,0.000,0.000,0.000,0.000,0.0,0.0,73.0,0.5,43.0,2.4,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0
1,132.0,0.006,0.000,0.006,0.003,0.0,0.0,17.0,2.1,0.0,10.4,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0
2,133.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.1,0.0,13.4,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0
3,134.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.4,0.0,23.0,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0
4,132.0,0.007,0.000,0.008,0.000,0.0,0.0,16.0,2.4,0.0,19.9,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140.0,0.000,0.000,0.007,0.000,0.0,0.0,79.0,0.2,25.0,7.2,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0
2122,140.0,0.001,0.000,0.007,0.000,0.0,0.0,78.0,0.4,22.0,7.1,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0
2123,140.0,0.001,0.000,0.007,0.000,0.0,0.0,79.0,0.4,20.0,6.1,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0
2124,140.0,0.001,0.000,0.006,0.000,0.0,0.0,78.0,0.4,27.0,7.0,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0


In [10]:
# data.shape
X.shape

(2126, 20)

In [12]:
input_layer_size  = 20
num_labels = 3

In [14]:
# Dividiendo los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Dividimos los datos para X_test y y_test donde seran los datos para el prueba
X_test = test_data.iloc[:, [0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19]]  # Seleccionamos los datos para la Xtest
y_test = test_data.iloc[:, 20]   # Seleccionamos los datos para la ytest

# Dividimos los datos para X_train y y_train para el entremamiento
X_train = train_data.iloc[:, [0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19]]
y_train = train_data.iloc[:,20]

m = y_train.size


# Mostramos los datos que seran para el entrenamiento
print("Datos de X_train:")
print(X_train)
print("Datos de y_train:")
print(y_train)

# Mostramos la cantidad de ejemplos que tienen X_train y y_train
print('-' * 80)
print("El 80% de ejemplos que seran para el entrenamiento son: {:.0f}".format(len(train_data)))
print("El 20% de ejemplos que seran para el test son: {:.0f}".format(len(test_data)))


Datos de X_train:
      baseline value  accelerations  fetal_movement  uterine_contractions  \
1233           125.0          0.000           0.000                 0.008   
480            140.0          0.000           0.000                 0.001   
1111           122.0          0.000           0.000                 0.000   
1303           137.0          0.005           0.000                 0.005   
861            142.0          0.003           0.000                 0.004   
...              ...            ...             ...                   ...   
1638           130.0          0.009           0.001                 0.004   
1095           123.0          0.012           0.000                 0.002   
1130           122.0          0.005           0.000                 0.004   
1294           115.0          0.003           0.000                 0.008   
860            142.0          0.001           0.000                 0.004   

      light_decelerations  severe_decelerations  prolongu

In [15]:
def calcularSigmoide(z):
    # Calcula la sigmoide de z.

    return 1.0 / (1.0 + np.exp(-z))

In [38]:

def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros, para un clasificador binario.
    Utilizado en One-vs-All para multiclase.

    Parametros
    ----------
    theta : array_like
        Parámetro theta de la regresión logística. Vector de la forma (n, ). n es el número de características
        incluida la intercepción.

    X : array_like
        Dataset con la forma (m, n). m es el número de ejemplos, y n es el número de
        características (incluida la intercepción).

    y : array_like
        El conjunto de etiquetas binarizadas para una clase. Un vector de la forma (m, ). m es el número de ejemplos.

    lambda_ : float
        Parámetro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la función de costo regularizada.

    grad : array_like
        Un vector de la forma (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta.
    """
    # Inicializa algunos valores útiles
    m = y.size  # número de ejemplos

    J = 0
    grad = np.zeros(theta.shape)

    # Calcula la hipótesis
    h = calcularSigmoide(X.dot(theta.T))

    # Para evitar regularizar el término theta_0
    temp = theta.copy()
    temp[0] = 0

    # Calcula el costo regularizado
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    # Calcula el gradiente regularizado
    grad = (1 / m) * (h - y).dot(X) + (lambda_ / m) * temp

    return J, grad

# Ejemplo de cómo utilizar esta función para One-vs-All:

def one_vs_all(X, y, num_labels, lambda_):
    """
    Entrena múltiples clasificadores binarios (uno contra todos) y devuelve todos
    los clasificadores entrenados en una matriz theta_all, donde la i-ésima fila
    corresponde al clasificador para la etiqueta i.

    Parametros
    ----------
    X : array_like
        Dataset de entrenamiento. (m x n)

    y : array_like
        Etiquetas de entrenamiento (m, ).

    num_labels : int
        Número de clases.

    lambda_ : float
        Parámetro de regularización.

    Devuelve
    -------
    theta_all : array_like
        Matriz donde cada fila corresponde a los parámetros entrenados para una clase.
    """
    m, n = X.shape
    theta_all = np.zeros((num_labels, n))

    # Añadir una columna de unos a X (para el término de intercepción)
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for i in range(1, num_labels + 1):
        # Binarizar las etiquetas para la clase actual (1 para la clase actual, 0 para las demás)
        y_i = np.where(y == i, 1, 0)

        # Inicializa theta
        initial_theta = np.zeros(n + 1)

        # Minimizar la función de costo para la clase actual
        # Puedes usar scipy.optimize.minimize para optimizar theta
        # Aquí te proporciono un ejemplo general, aunque puedes integrar scipy para hacer la optimización:
        J, grad = lrCostFunction(initial_theta, X, y_i, lambda_)
        theta_all[i - 1, :] = grad  # Cambiar grad por el valor optimizado

    return theta_all


In [39]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)

    sigma[sigma == 0] = 1  # verificar

    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [40]:
X_norm, mu, sigma = featureNormalize(X_train)
X_norm

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance
1233,-0.818907,-0.824976,-0.204261,1.218555,-0.637350,-0.064301,-0.268148,-0.262589,-0.367038,0.829032,-0.500418,0.090151,-0.947494,-0.014607,-0.445313,-0.680274,-0.603307,-0.742903,-0.580514
480,0.692429,-0.824976,-0.204261,-1.159016,-0.637350,-0.064301,-0.268148,-0.604484,1.233861,0.513834,1.267146,-0.482204,1.975298,1.686204,-0.445313,0.222228,0.414302,0.281431,-0.545964
1111,-1.121174,-0.824976,-0.204261,-1.498669,-0.296949,-0.064301,-0.268148,0.535166,-0.532648,0.969120,-0.807820,0.325826,-1.228531,-1.035093,-0.445313,-1.041275,-0.984911,-1.152636,-0.545964
1303,0.390162,0.464049,-0.204261,0.199596,0.043451,-0.064301,-0.268148,-0.490519,-0.532648,-0.571847,-0.193016,0.729841,0.794940,-0.014607,-0.445313,0.643396,0.859505,0.759453,-0.338667
861,0.893941,-0.051561,-0.204261,-0.140057,-0.637350,-0.064301,-0.268148,-0.718449,0.957844,-0.414248,-1.140840,1.335863,-0.273003,-1.035093,-0.445313,0.763729,0.859505,0.827742,-0.615063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,-0.315128,1.495268,-0.183561,-0.140057,-0.296949,-0.064301,-0.268148,-0.034659,-0.532648,-1.027133,0.729192,-0.684211,0.457695,0.665717,-0.445313,0.402728,0.414302,0.418008,-0.096820
1095,-1.020419,2.268683,-0.204261,-0.819363,-0.637350,-0.064301,-0.268148,0.991026,-0.532648,-0.274160,-0.474801,0.224822,-0.666456,-0.694931,-0.445313,-0.379440,-0.158104,-0.333169,-0.511415
1130,-1.121174,0.464049,-0.204261,-0.140057,1.064653,-0.064301,-0.268148,1.446887,-0.532648,1.652049,0.703575,-1.121894,-0.329211,0.665717,-0.445313,-0.379440,-0.857710,-0.811192,0.421423
1294,-1.826465,-0.051561,-0.204261,1.218555,0.043451,-0.064301,1.383487,0.307236,-0.532648,0.321213,0.959744,-0.751547,0.851147,-0.354769,2.444122,-0.259107,-0.794109,-0.606325,0.905117


In [41]:
m, n = X_norm.shape

print(f"m (número de filas): {m}")
print(f"n (número de columnas): {n}")

m (número de filas): 1700
n (número de columnas): 19


In [42]:
X_ready = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X_ready.shape

(1700, 20)

In [43]:
# valores de prueba para los parámetros theta
theta_t = np.zeros(20)
# valores de testeo para el parametro de regularizacion
lambda_t = 0.1
J, grad = lrCostFunction(theta_t, X_ready, y_train, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('-----------------------')

Costo         : 0.693147
-----------------------


In [44]:
def oneVsAll(X, y, num_labels, lambda_):

    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)


    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(calcularCosto,
                                initial_theta,
                                (X, (y == (c + 1)), lambda_),
                                jac=True,
                                method='BFGS',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [45]:
all_theta = oneVsAll(X_ready, y_train, num_labels, lambda_t)

In [46]:
print(all_theta)

[[-1.67562999e+00 -1.57961446e+00  4.50740560e-02 -3.46328980e-01
   4.21306836e-02  1.80888368e-01  2.18988677e-01  1.20487158e+00
   9.89588460e-01  9.01553092e-02 -1.38315592e-03 -2.84634332e-01
   1.81282982e+00 -1.11358654e+01 -1.46133475e+01 -3.08511172e-01
  -1.36198243e-01  2.27542251e+00  4.26184701e+00  1.15146379e+01
   3.53014835e-01]
 [-3.00718753e+01 -1.35267886e-02 -2.18111757e-16 -3.37742822e-16
  -2.85346294e-15  4.92527410e-16 -1.08349697e-15 -1.63357732e-15
   3.40314160e-15  3.14576399e-15  1.22362054e-15 -2.69173216e-15
   2.61695403e-15  9.13828699e-16  7.48908440e-15 -7.28934912e-16
  -5.41796573e-16 -6.06316253e-15 -5.84776929e-15 -4.40783998e-15
   7.59387235e-16]
 [-3.00718753e+01 -1.35267886e-02 -2.18111757e-16 -3.37742822e-16
  -2.85346294e-15  4.92527410e-16 -1.08349697e-15 -1.63357732e-15
   3.40314160e-15  3.14576399e-15  1.22362054e-15 -2.69173216e-15
   2.61695403e-15  9.13828699e-16  7.48908440e-15 -7.28934912e-16
  -5.41796573e-16 -6.06316253e-15 -5.8

In [47]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(calcularSigmoide(X.dot(all_theta.T)), axis = 1)

    return p + 1

In [48]:
print(X_ready.shape)
print(all_theta.shape)
pred = predictOneVsAll(all_theta, X_ready)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y_train) * 100))

(1700, 20)
(3, 21)
Precision del conjuto de entrenamiento: 39.47%


In [49]:
# Normalizamos el X_test
X_test_norm = (X_test - mu) / sigma
m_test = len(X_test)

X_test_norm

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance
282,-0.012861,-0.309366,0.002740,-0.479710,0.043451,-0.064301,-0.268148,-0.262589,-0.532648,1.249296,-0.039314,0.056483,0.008034,0.325555,-0.445313,0.101894,0.032698,0.008275,-0.338667
1999,-0.818907,-0.824976,-0.183561,1.558208,2.085854,-0.064301,-0.268148,0.421201,-0.532648,-1.254776,0.037536,-0.852550,-1.340946,0.325555,-0.445313,-0.439607,-1.175713,-0.879481,0.352324
1709,-0.214373,0.206244,-0.142161,-0.140057,1.064653,-0.064301,1.383487,0.877061,-0.532648,-1.289798,0.498640,-0.515871,0.232864,1.346042,-0.445313,-0.259107,-0.476106,-0.401458,0.075928
988,-0.214373,2.010878,-0.204261,0.199596,-0.637350,-0.064301,-0.268148,-0.034659,-0.532648,-0.659402,0.473023,-0.381200,0.401487,1.346042,-0.445313,0.342562,0.668704,0.486297,-0.338667
2018,-0.818907,-0.824976,-0.204261,1.218555,1.745453,-0.064301,1.383487,-0.034659,-0.532648,-0.992111,0.165621,-0.515871,-0.497833,-0.014607,-0.445313,-1.402276,-1.493715,-1.630658,-0.407766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,1.196208,-0.824976,0.230441,-1.498669,-0.637350,-0.064301,-0.268148,-1.174309,1.123455,0.041038,-0.423567,1.268527,1.188392,-0.694931,-0.445313,0.462895,0.605103,0.554586,-0.615063
973,-0.214373,-0.824976,-0.204261,0.199596,2.085854,-0.064301,-0.268148,0.535166,-0.201428,-1.044644,0.677958,-1.155561,-0.441626,-0.354769,0.999405,-0.138773,-0.921310,-0.333169,1.768856
611,1.095452,0.721854,-0.100761,-0.819363,-0.637350,-0.064301,-0.268148,0.421201,-0.532648,2.317467,0.831659,-0.751547,0.570110,0.325555,0.999405,0.763729,0.923106,0.896031,-0.304117
2108,0.289406,-0.824976,-0.142161,1.218555,-0.296949,-0.064301,-0.268148,0.991026,-0.532648,-0.676913,-0.654119,0.224822,-1.059909,-0.354769,-0.445313,-0.259107,-0.221704,-0.128303,-0.580514


In [50]:
X_test_ready = np.concatenate([np.ones((m_test, 1)), X_test_norm], axis=1)
print(X_test_ready.shape)
print(all_theta.shape)

(426, 20)
(3, 21)


In [51]:
# Inicializamos nuestra y_predicha donde almacenaremos nuestras y predichas
# En lugar de usar un loop, usa la función predictOneVsAll directamente
y_predicha = predictOneVsAll(all_theta, X_test_ready)

# Comparar con los datos reales
# Verificamos la forma de los arrays
print("Forma de y_predicha:", y_predicha.shape)
print("Forma de y_test:", y_test.shape)

# Comparar las predicciones con las etiquetas reales
accuracy = np.mean(y_predicha == y_test)  # Calcular la precisión
print(f'Precisión: {accuracy * 100:.2f}%')


Forma de y_predicha: (426,)
Forma de y_test: (426,)
Precisión: 41.08%
